# Simple RAG Application using LlamaIndex

### Table of Contents

1. **Load "llama2" Model**
2. **Load External Data**
3. **Build Index**
4. **Build Complete RAG Pipeline**

### Installation

* **pip install llama-index**
* **pip install llama-index-core llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface**

## 1. Load llama2 Model

In [2]:
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama2", request_timeout=600) #base_url = 'http://localhost:11434',

In [3]:
response = llm.complete("Do you know about Claude-3?") ## stream_complete() for streaming response

response

CompletionResponse(text='I\'m not familiar with a "Claude-3." Could you please provide more context or information about what Claude-3 is? Is it a person, place, or thing? Additionally, what is the relevance of Claude-3 to your inquiry?', additional_kwargs={'model': 'llama2', 'created_at': '2024-03-08T05:43:28.61649818Z', 'done': True, 'context': [518, 25580, 29962, 3532, 14816, 29903, 29958, 5299, 829, 14816, 29903, 6778, 13, 13, 6132, 366, 1073, 1048, 19108, 29899, 29941, 29973, 518, 29914, 25580, 29962, 13, 29902, 29915, 29885, 451, 9985, 411, 263, 376, 20216, 1151, 29899, 29941, 1213, 6527, 366, 3113, 3867, 901, 3030, 470, 2472, 1048, 825, 19108, 29899, 29941, 338, 29973, 1317, 372, 263, 2022, 29892, 2058, 29892, 470, 2655, 29973, 19814, 29892, 825, 338, 278, 29527, 749, 310, 19108, 29899, 29941, 304, 596, 25501, 16129, 29973], 'total_duration': 8422157117, 'load_duration': 249350, 'prompt_eval_duration': 186733000, 'eval_count': 55, 'eval_duration': 8234642000}, raw={'model': 'lla

In [4]:
print(response.text)

I'm not familiar with a "Claude-3." Could you please provide more context or information about what Claude-3 is? Is it a person, place, or thing? Additionally, what is the relevance of Claude-3 to your inquiry?


## 2. Load External Documents

* **pip install -U bs4 requests**

In [5]:
import requests
from bs4 import BeautifulSoup
from llama_index.core import Document

urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ]

docs = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    docs.append(Document(text=soup.text, metadata={"source": url}))

In [26]:
docs[-2]

Document(id_='1787d631-8b7b-4f46-a12c-1b2f15d76fde', embedding=None, metadata={'source': 'https://www.anthropic.com/news/claude-3-family'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Introducing the next generation of Claude \\ AnthropicClaudeAPIResearchCompanyNewsCareersProductAnnouncementsIntroducing the next generation of ClaudeMar 4, 2024●7 min readTry Claude 3Today, we\'re announcing the Claude 3 model family, which sets new industry benchmarks across a wide range of cognitive tasks. The family includes three state-of-the-art models in ascending order of capability: Claude 3 Haiku, Claude 3 Sonnet, and Claude 3 Opus. Each successive model offers increasingly powerful performance, allowing users to select the optimal balance of intelligence, speed, and cost for their specific application.Opus and Sonnet are now available to use in claude.ai and the Claude API which is now generally available in 159 countries. Haiku will be available soon

## 3. Build Index

In [6]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents(docs)

In [22]:
retriever = index.as_retriever()

relevant_docs = retriever.retrieve("Do you know about Claude-3?")

relevant_docs

[NodeWithScore(node=TextNode(id_='abfbcab8-72a7-4f60-92b7-125852b402c3', embedding=None, metadata={'source': 'https://www.anthropic.com/news/claude-3-family'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1787d631-8b7b-4f46-a12c-1b2f15d76fde', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'source': 'https://www.anthropic.com/news/claude-3-family'}, hash='a0d12e9f4fabd8677bc7b8261c6038ffcb44812f5040d50ff3cdfa3ce14cda08'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='20e87106-b7d9-43ba-b9ea-bb6ffed1c1f0', node_type=<ObjectType.TEXT: '1'>, metadata={'source': 'https://www.anthropic.com/news/claude-2-1-prompting'}, hash='2405a2348b0251f598512139ff7c12cb01e861b353e1f003b0365a6a9c48e913'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='393bb60a-6e55-4f05-b19e-76e04b3d1dd2', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='e6c7c2983e3df6abc91ecbaead79d494b2e12fbcd599914a384de

In [24]:
for doc in relevant_docs:
    print(f"Doc: {doc.metadata}")

Doc: {'source': 'https://www.anthropic.com/news/claude-3-family'}
Doc: {'source': 'https://www.anthropic.com/claude'}


## 4. Build Complete RAG Pipeline

In [27]:
from llama_index.core import Settings

Settings.llm = Ollama(model="llama2", request_timeout=600)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

index = VectorStoreIndex.from_documents(docs)

query_engine = index.as_query_engine() # similarity_top_k, streaming=True

query_engine

In [7]:
response = query_engine.query("Do you know about Claude-3?")

print(response)

Yes, I am familiar with Claude-3. According to the provided context information, Claude-3 is a family of foundational AI models that can be used in various applications such as brainstorming ideas, analyzing images, and processing long documents. The Claude-3 family includes three state-of-the-art models, Haiku, Sonnet, and Opus, which offer increasingly powerful performance and are available through the Claude API. These models are designed to provide fast and accurate responses, with a 200K context window initially offered upon launch. Additionally, the Claude-3 models have sophisticated vision capabilities, fewer refusals, improved accuracy, and long context and near-perfect recall capabilities.


In [9]:
type(response)

llama_index.core.base.response.schema.Response

In [10]:
response.metadata

{'abfbcab8-72a7-4f60-92b7-125852b402c3': {'source': 'https://www.anthropic.com/news/claude-3-family'},
 '88d76dcc-b61b-419e-8492-1ea214cd0323': {'source': 'https://www.anthropic.com/claude'}}

In [16]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='abfbcab8-72a7-4f60-92b7-125852b402c3', embedding=None, metadata={'source': 'https://www.anthropic.com/news/claude-3-family'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1787d631-8b7b-4f46-a12c-1b2f15d76fde', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'source': 'https://www.anthropic.com/news/claude-3-family'}, hash='a0d12e9f4fabd8677bc7b8261c6038ffcb44812f5040d50ff3cdfa3ce14cda08'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='20e87106-b7d9-43ba-b9ea-bb6ffed1c1f0', node_type=<ObjectType.TEXT: '1'>, metadata={'source': 'https://www.anthropic.com/news/claude-2-1-prompting'}, hash='2405a2348b0251f598512139ff7c12cb01e861b353e1f003b0365a6a9c48e913'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='393bb60a-6e55-4f05-b19e-76e04b3d1dd2', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='e6c7c2983e3df6abc91ecbaead79d494b2e12fbcd599914a384de

## Summary

In this video, I explained how to create simple **RAG** application using **llamaindex**. Feel free to let me know your views and doubts in comments section.